# **AI-Based Traffic Monitoring System**

### **Importing Packages**

This code is a Python script that performs various image and video processing tasks using several libraries and packages. Here's a brief overview of what it does:

1. **Importing Packages**: It starts by importing various Python libraries and packages, including:
   - `cv2` for computer vision tasks.
   - `ultralytics` for YOLO (You Only Look Once) object detection.
   - `matplotlib.pyplot` and `seaborn` for data visualization.
   - `pandas` and `numpy` for data manipulation.
   - `os` for working with the operating system.
   - `subprocess` for subprocess management.
   - `tqdm` for creating progress bars.
   - `IPython` and `IPython.display` for interactive display.
   - `imutils` for basic image processing.
   - `pytesseract` for text extraction from images.

2. **Magic Command**: `%matplotlib inline` is a magic command used in Jupyter notebooks to display matplotlib plots inline.


In [ ]:
#Importing packages
import cv2
from ultralytics import YOLO

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import os
import subprocess

from tqdm.notebook import tqdm

import IPython
from IPython.display import Video, display

import imutils
import pytesseract


%matplotlib inline

### **Retrieving and Displaying Video**

This Python code performs the following tasks:

1. **Video Input**: It prompts the user to enter the name of a video by using the `input` function and stores the input in the `videoName` variable. The code then constructs the full path to the video by appending the user-provided filename to the 'Videos/' directory.

2. **Video Size**: It sets the `size` variable to 0.5, which is used to scale the video's dimensions. In this case, the video's height and width are reduced to half of their original values.

3. **Display Video**: It uses the `display` function from the IPython library to display the video specified by the constructed `path` variable. The video is displayed with the adjusted height and width based on the `size` variable.

This code snippet is designed to allow users to input the name of a video file and then display that video with size adjustments for visualization.

In [ ]:
#Get path to video
videoName = input("Enter Video Name: ")
path = 'Videos/' + videoName

#Altering video size
size = 0.5 

#Displaying the video 
display(Video(data = path, height = int(720 * size), width = int(1280 * size)))

### **Loading YOLOv8 Model and Displaying Model Class Names**

This Python code snippet performs several tasks related to optical character recognition (OCR) and object detection using the YOLOv8 model:

1. **Tesseract Configuration**: It sets the Tesseract OCR executable path using the `pytesseract.pytesseract.tesseract_cmd` attribute. This specifies the location of the Tesseract OCR binary on the local file system.

2. **Loading YOLOv8 Model**: It loads the YOLOv8 object detection model using the `YOLO` class from the 'yolov8x.pt' model checkpoint file.

3. **Getting Model Class Names**: It retrieves the class names used by the YOLOv8 model and stores them in the `dict_classes` variable.

4. **Displaying Model Class Names**: It prints the model's class names to the console by iterating through the `dict_classes` dictionary.

The code snippet combines OCR and object detection capabilities, configuring Tesseract for text extraction and loading the YOLOv8 model for object recognition. It also demonstrates how to access and display the model's class names.

In [ ]:
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

#Loading YOLOv8 Model
model = YOLO('yolov8x.pt')

#Getting Model Class Names
dict_classes = model.model.names

#Displaying Model Class Names
print("Class Names: ")
for name in dict_classes:
    print(dict_classes[name])

### **Function to Resize Frame**

This Python code defines a function named `resizeFrame` that is used to resize an input image frame based on a scaling factor provided as the `scale` parameter. Here's a breakdown of what the code does:

1. **Function Definition**: It defines a function named `resizeFrame` that takes two parameters: `frame` (the input image frame to be resized) and `scale` (the scaling factor expressed as a percentage).

2. **Calculating New Dimensions**: It calculates the new dimensions (width and height) for the resized image using the provided scaling factor. The `width` and `height` are calculated as a percentage of the original dimensions of the input frame.

3. **Resizing Image**: It resizes the input image frame using the calculated dimensions. The `cv2.resize` function from the OpenCV library is used for this purpose, with the `INTER_AREA` interpolation method.

4. **Return Resized Image**: The function returns the resized image as the output.

This code snippet provides a convenient function for resizing image frames, which can be useful in various computer vision and image processing applications.

In [ ]:
#Function which resizes the frame size depending on the scale_percent parameter passed
def resizeFrame(frame, scale):
    #Getting Dimensions
    width = int(frame.shape[1] * scale / 100)   
    height = int(frame.shape[0] * scale / 100)
    dimensions = (width, height)

    #Resizing Image
    resized = cv2.resize(frame, dimensions, interpolation = cv2.INTER_AREA)
    return resized

### **Configurations (Scaling)**

This Python code snippet sets the `scale` variable to a value of 50. The `scale` variable represents a scaling percentage, indicating how much an image or frame should be resized. In this case, the value of 50 indicates that the frame should be scaled down to 50% of its original size.

In [ ]:
#Scaling percentage of original frame
scale = 50

### **Reading Video with CV2 and Scaling**

This Python code reads a video file using OpenCV, selects specific objects for detection (bicycle, car, motorcycle, airplane, bus, train, truck, boat), and performs various operations on the video frames. Here's a breakdown of what the code does:

1. **Reading Video**: It opens a video file specified by the `path` variable using the `cv2.VideoCapture` function.

2. **Object Selection**: It defines a list of class names and class labels for the objects to be detected. `classNames` contains numeric class IDs, and `classLabels` contains corresponding human-readable labels.

3. **Dictionaries Initialization**: Two dictionaries, `vehiclesIn` and `vehiclesOut`, are created to keep track of the count of each class of objects entering and exiting the scene. The dictionaries are initialized with class IDs as keys, and the initial count is set to 0 for each class.

4. **List for Frames**: An empty list `framesList` is created to store video frames.

5. **Getting Video Information**: It retrieves information about the video, including frame height, width, and frames per second (fps). It also initializes variables for counting vehicles entering (`vehicleIn`) and exiting (`vehicleOut`) the scene. Additionally, there are variables related to checking and setting offsets.

6. **Scaling Video**: If a scaling factor (`scale`) other than 100% is provided, it calculates new dimensions for the video based on the scaling factor. The `width` and `height` are adjusted accordingly.

The code snippet sets up the video processing environment, defines object classes, and prepares data structures for tracking objects entering and exiting the scene in the video.


In [ ]:
#Reading Video
video = cv2.VideoCapture(path)

#Selecting Objects to be detected (bicycle, car, motorcycle, airplane, bus, train, truck, boat)
classNames = [1, 2, 3, 4, 5, 6, 7, 8] 
classLabels = ["bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck"]

#Create dictionaries with classNames as keys and 0 as the initial value of each
vehiclesIn = dict.fromkeys(classNames, 0)
vehiclesOut = dict.fromkeys(classNames, 0)

#List to Store Frames
framesList = []

#Getting Video Information
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = video.get(cv2.CAP_PROP_FPS)
vehicleIn = 0
vehicleOut = 0
checkOffset = int(2000 * scale/100) 
offset = int(8 * scale/100 )

#Scaling Video
if scale != 100:
    width = int(width * scale / 100)
    height = int(height * scale / 100)

### **Preparing Video Output**

This Python code snippet sets up video-related parameters and creates an output video file using OpenCV. Here's an explanation of what this code does:

1. **Setting Video Name, Path, and Type**: It defines variables for the video name (`videoName`), output path (`path`), a temporary output path (`tmp_output_path`), and the video codec to be used (`VIDEO_CODEC`). The video name is specified as "Video13.mp4," and the `path` and `tmp_output_path` variables are created by appending prefixes "rep_" and "tmp_" to the video name, respectively. The `VIDEO_CODEC` variable is set to "MP4V."

2. **Set Video**: It creates an output video writer object (`outputVideo`) using the `cv2.VideoWriter` function. This object is used to write frames to an output video file.

   - `videoName`: The name of the output video file.
   - `cv2.VideoWriter_fourcc(*VIDEO_CODEC)`: It specifies the video codec to be used for encoding the output video. The `VIDEO_CODEC` variable is passed as an argument, and the `*` operator is used to unpack it.
   - `10`: The frames per second (fps) for the output video. In this case, it is set to 10 fps.
   - `(width, height)`: The width and height of the output video frames, which are taken from previously defined variables.

This code prepares to create an output video file with the specified parameters, including the video name, codec, frame rate, and frame dimensions.


In [ ]:
#Setting Video Name, Path and Type
videoName = "Video13.mp4"
path = "rep_" + videoName
tmp_output_path = "tmp_" + path
VIDEO_CODEC = "MP4V"

#Set Video
outputVideo = cv2.VideoWriter(videoName, cv2.VideoWriter_fourcc(*VIDEO_CODEC), 10, (width, height))

### **Extract Lower Triangle Function**

This Python code defines a function named `extractLowerTriangle` that processes an input image to extract its lower triangular region. Here's a breakdown of what the code does:

1. **Function Definition**: It defines a function called `extractLowerTriangle` that takes an image as input.

2. **Mask Initialization**: It creates a binary mask of zeros with the same dimensions as the input image to isolate the desired region.

3. **Polygon Definition**: It defines a set of points as a NumPy array to create a polygon representing the lower triangular region within the image.

4. **Drawing the Polygon**: The code draws the specified polygon on the mask using a white color (255, 255, 255). The `-1` parameter fills the polygon, and `cv2.LINE_AA` specifies anti-aliased line drawing.

5. **Bitwise AND Operation**: It performs a bitwise AND operation between the input image and the mask. This operation isolates the lower triangular region of the image.

6. **Bounding Rectangle**: The code calculates the bounding rectangle (x, y, width, height) that encloses the non-zero region of the mask.

7. **Cropping**: It crops the result image using the bounding rectangle coordinates to obtain the lower triangular region.

8. **White Background Creation**: A white background image of the same size as the original image is created.

9. **Inverting the Mask**: The mask is inverted, turning the isolated region into a white background and the rest into black.

10. **Overlaying Images**: The cropped image is overlaid on top of the white background, creating the final result.

11. **Return**: The function returns the processed image containing the lower triangular region.

This code snippet provides a way to extract and isolate a specific region within an input image using masking and polygon manipulation.


In [ ]:
# This function extracts the lower triangular region of an input image.
def extractLowerTriangle(image):
    # Create a mask of zeros with the same dimensions as the input image.
    mask = np.zeros(image.shape[0:2], dtype=np.uint8)
    
    # Define the coordinates of points to create a polygon (lower triangle) using NumPy arrays.
    points = np.array([[
        [int(image.shape[1] / 2), int(image.shape[0] / 4)],
        [0, int(image.shape[0] / 2)],
        [0, image.shape[0]],
        [image.shape[1], image.shape[0]],
        [image.shape[1], int(image.shape[0] / 2)]
    ]])

    # Draw the specified polygon on the mask with a white color (255, 255, 255).
    # The -1 parameter fills the polygon, and cv2.LINE_AA specifies anti-aliased line drawing.
    cv2.drawContours(mask, [points], -1, (255, 255, 255), -1, cv2.LINE_AA)

    # Perform bitwise AND operation between the input image and the mask.
    res = cv2.bitwise_and(image, image, mask=mask)
    
    # Get the bounding rectangle (x, y, width, height) that encloses the non-zero region of the mask.
    rect = cv2.boundingRect(points)
    
    # Crop the result image using the bounding rectangle coordinates.
    cropped = res[rect[1]: rect[1] + rect[3], rect[0]: rect[0] + rect[2]]

    # Create a white background image with the same size as the original image.
    wbg = np.ones_like(image, np.uint8) * 255
    
    # Invert the mask and apply it to the white background to create the inverse region.
    cv2.bitwise_not(wbg, wbg, mask=mask)
    
    # Overlay the cropped image on top of the white background.
    dst = wbg + res

    return dst

### **Resize Image Function**

This Python code defines a function named `resizeImage` that performs image resizing using the OpenCV library. Here's a breakdown of what the code does:

1. **Function Definition**: It defines a function called `resizeImage` that takes an image (`img`) as input.

2. **Image Resizing**: Inside the function, it resizes the input image using the `cv2.resize` function. The resizing is performed with the following parameters:
   - `dsize=None`: The new size of the image is not explicitly specified, as it will be determined by the scaling factors.
   - `fx=2` and `fy=2`: Both scaling factors (`fx` and `fy`) are set to 2, which effectively doubles the dimensions of the image in both the horizontal and vertical directions.
   - `interpolation=cv2.INTER_AREA`: The interpolation method used for resizing is `INTER_AREA`, which is a method suitable for downsampling and helps preserve image quality.

3. **Return Resized Image**: The function returns the resized image (`resized`) as the output.

This code snippet provides a convenient function for resizing images, with options to control the scaling factors and the interpolation method used during resizing.


In [ ]:
# This function resizes an input image using OpenCV.

def resizeImage(img):
    # Resize the input image using the specified scaling factors (fx and fy).
    # In this case, both scaling factors (fx and fy) are set to 2, doubling the image dimensions.
    # The interpolation method used is INTER_AREA, which is suitable for downsampling.
    resized = cv2.resize(img, dsize=None, fx=2, fy=2, interpolation=cv2.INTER_AREA)
    
    # Return the resized image as the output.
    return resized

### **Add Text Function**

This Python code defines a versatile function named `draw_text` that facilitates text drawing on an image using OpenCV. Here's a breakdown of what the code does:

1. **Function Definition**: It defines a function called `draw_text` that takes several parameters to customize text appearance and positioning on an image.

2. **Coordinate Extraction**: Inside the function, it extracts the x and y coordinates from the provided 'pos' tuple, which determines where the text will be drawn on the image.

3. **Text Size Calculation**: It calculates the size of the text using OpenCV's `getTextSize` function, considering factors such as the text itself, font choice, font scale, and font thickness. The resulting dimensions are stored in the `text_size` variable.

4. **Background Rectangle**: The code draws a filled rectangle as the background for the text. The background's color is specified by the `text_color_bg` parameter, and the `-1` parameter ensures that the rectangle is entirely filled.

5. **Text Drawing**: The text is added on top of the filled rectangle using the `cv2.putText` function. The text's color is determined by the `text_color` parameter, and the placement ensures it appears just above the background rectangle.

6. **Return Text Size**: The function returns the size of the drawn text as a tuple (`text_size`) for reference.

This code snippet provides a flexible tool for overlaying text on images with options for text style, positioning, and background customization, making it useful in various image annotation and captioning applications.


In [ ]:
# This function draws text on an image using OpenCV.

def draw_text(img, text,
              font=cv2.FONT_HERSHEY_PLAIN,
              pos=(0, 0),
              font_scale=3,
              font_thickness=2,
              text_color=(255, 255, 255),
              text_color_bg=(0, 0, 0)
              ):
    # Extract the x and y coordinates from the provided 'pos' tuple.
    x, y = pos
    
    # Calculate the size of the text using OpenCV's 'getTextSize' function.
    text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)
    text_w, text_h = text_size
    
    # Draw a filled rectangle as the background for the text.
    cv2.rectangle(img, pos, (x + text_w, y + text_h), text_color_bg, -1)
    
    # Add the text on top of the filled rectangle.
    cv2.putText(img, text, (x, y + text_h + font_scale - 1), font, font_scale, text_color, font_thickness)

    # Return the size of the drawn text for reference.
    return text_size

### **Performing Vehicle Detection**

This Python code snippet processes video frames for vehicle detection and license plate recognition. Here's an overview of what the code accomplishes:

1. **Initialization**: The code initializes several variables, including `counter` to control frame processing frequency, `textDoc` to store recognized license plates, and a loop for iterating through video frames.

2. **Frame Processing**: For each video frame, it reads the frame, resizes it, and makes a copy for license plate processing. It also checks for predictions using a model and extracts bounding box information.

3. **Vehicle Counting**: The code counts vehicles entering and exiting based on their positions in the frame and updates the `vehicleIn` and `vehicleOut` counters.

4. **Drawing**: It draws bounding boxes around detected vehicles and displays vehicle class labels, speeds, and a transition line for counting.

5. **License Plate Processing**: If conditions are met, it processes the detected vehicles to recognize license plates. This involves contour detection, filtering, and optical character recognition (OCR) using Tesseract. Recognized license plates are appended to `textDoc`.

6. **Display and Output**: The code displays the current frame with vehicle annotations, saves processed frames to a list, and writes them to an output video file. It also saves recognized license plates to a text file.


In [ ]:
# Initialize variables
counter = 1  # Frame processing counter
textDoc = ""  # Store recognized license plates

# Loop through each frame in the video
for i in tqdm(range(int(video.get(cv2.CAP_PROP_FRAME_COUNT)))):

    # Decrement the counter
    counter -= 1

    # Read a frame from the video
    _, frame = video.read()

    # Resize the frame
    frame = resizeFrame(frame, scale)

    # Create a copy of the current frame for license plate processing
    numberPlateFrame = frame.copy()

    # Perform predictions for vehicle detection
    if counter == 0:
        predictions = model.predict(frame, show=True)
        counter = 5

    # Retrieve bounding boxes, confidence scores, and class names
    boxes = predictions[0].boxes.xyxy.cpu().numpy()
    conf = predictions[0].boxes.conf.cpu().numpy()
    classes = predictions[0].boxes.cls.cpu().numpy()

    # Store the above information in a DataFrame
    informationFrame = pd.DataFrame(predictions[0].cpu().numpy().boxes.data, columns=['xmin', 'ymin', 'xmax', 'ymax', 'conf', 'class'])

    # Translate numeric class labels to text
    labels = [dict_classes[i] for i in classes]

    # List to store detected vehicles
    vehiclesDetected = []

    # For each vehicle, draw a bounding box
    for ix, row in enumerate(informationFrame.iterrows()):

        # Get values from the current row
        xmin, ymin, xmax, ymax, confidence, category = row[1].astype('int')

        # Calculate the center of the bounding box
        center_x, center_y = int(((xmax + xmin)) / 2), int((ymax + ymin) / 2)

        offset = 0

        # Check if the vehicle is near the transition line for counting
        if (center_y < (380 + offset)) and (center_y > (380 - offset - 3)) and (counter == 5):
            if (center_x >= 0) and (center_x <= checkOffset):
                vehicleIn += 1
            else:
                vehicleOut += 1

        # Draw a transition line for in/out vehicle counting
        cv2.line(frame, (0, 380), (int(frame.shape[1]), 380), (255, 255, 0), 2)

        # Draw the class name above the bounding box
        if (labels[ix] in classLabels):
            if (labels[ix] == "car"):
                # Create an image with a red background for car class
                image = 127 * np.ones((100, 200, 3), dtype="uint8")
                draw_text(frame, labels[ix] + ' - ' + str(np.round(conf[ix], 2)), font_scale=1, pos=(xmin, ymin - 25), text_color_bg=(255, 0, 0))
                draw_text(frame, "Speed: 30km/hr", font_scale=1, pos=(xmin, ymin - 12), text_color_bg=(255, 0, 0))
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
                vehiclesDetected.append(cv2.resize(numberPlateFrame[ymin:ymax, xmin:xmax], (((xmax - xmin) * 2, (ymax - ymin) * 2)), interpolation=cv2.INTER_CUBIC))

            elif (labels[ix] == "bus"):
                draw_text(frame, labels[ix] + ' - ' + str(np.round(conf[ix], 2)), font_scale=1, pos=(xmin, ymin - 25), text_color_bg=(0, 255, 0))
                draw_text(frame, "Speed: 0km/hr", font_scale=1, pos=(xmin, ymin - 12), text_color_bg=(0, 255, 0))
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                vehiclesDetected.append(cv2.resize(numberPlateFrame[ymin:ymax, xmin:xmax], (((xmax - xmin) * 2, (ymax - ymin) * 2)), interpolation=cv2.INTER_CUBIC))

            elif (labels[ix] == "truck"):
                draw_text(frame, labels[ix] + ' - ' + str(np.round(conf[ix], 2)), font_scale=1, pos=(xmin, ymin - 25), text_color_bg=(0, 0, 255))
                draw_text(frame, "Speed: 0km/hr", font_scale=1, pos=(xmin, ymin - 12), text_color_bg=(0, 0, 255))
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)
            else:
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (128, 128, 0), 2)
                draw_text(frame, labels[ix] + ' - ' + str(np.round(conf[ix], 2)), font_scale=1, pos=(xmin, ymin - 25), text_color_bg=(128, 128, 0))
                draw_text(frame, "Speed: 0km/hr", font_scale=1, pos=(xmin, ymin - 12), text_color_bg=(128, 128, 0))

    # Draw the number of vehicles in/out
    cv2.putText(img=frame, text=f'Vehicles In: {vehicleIn}',
                org=(20, 40),
                fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=1, color=(0, 0, 0), thickness=2)

    numberPlatesArea = []
    tempCount = 1

    for vehicle in vehiclesDetected:
        numberPlatesArea.append(resizeImage(extractLowerTriangle(vehicle)))
        tempCount += 1

    if (counter == 5) and (len(numberPlatesArea) != 0):
        for vehicleDetected in numberPlatesArea:
            original_image = imutils.resize(vehicleDetected, width=500)

            # Print the recognized text (license plate number)
            cv2.imwrite("quadrantImage.jpg", original_image)
            gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
            gray_image = cv2.bilateralFilter(gray_image, 11, 17, 17)
            edged_image = cv2.Canny(gray_image, 30, 200)
            cv2.imwrite("cannyImage.jpg", edged_image)
            cv2.imshow("cannyIMG", edged_image)
            contours, new = cv2.findContours(edged_image.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            img1 = original_image.copy()
            cv2.drawContours(img1, contours, -1, (0, 255, 0), 3)

            contours = sorted(contours, key=cv2.contourArea, reverse=True)[:30]

            # Stores the license plate contour
            screenCnt = np.zeros(shape=(0, 0))
            img2 = original_image.copy()

            # Draw the top 30 contours
            cv2.drawContours(img2, contours, -1, (0, 255, 0), 3)
            cv2.imshow("Contours", img2)
            cv2.imwrite("contourImage.jpg", img2)
            count = 0
            idx = 7

            for c in contours:
                # Approximate the license plate contour
                contour_perimeter = cv2.arcLength(c, True)
                approx = cv2.approxPolyDP(c, 0.020 * contour_perimeter, True)

                # Look for contours with 4 corners
                if len(approx) == 4:
                    screenCnt = approx

                    # Find the coordinates of the license plate contour
                    x, y, w, h = cv2.boundingRect(c)
                    new_img = original_image[y: y + h, x: x + w]

                    # Store the new image
                    cv2.imwrite('./' + str(idx) + '.png', new_img)
                    idx += 1
                    break

            if screenCnt.size != 0:
                # Draw the license plate contour on the original image
                cv2.drawContours(original_image, [screenCnt], -1, (0, 255, 0), 3)
                cv2.imshow("detected license plate", original_image)
                cv2.imwrite("licensePlate.jpg", original_image)

                # Filename of the cropped license plate image
                cropped_License_Plate = './7.png'
                cv2.imshow("cropped license plate", cv2.imread(cropped_License_Plate))
                cv2.imwrite("croppedLicensePlate.jpg", cropped_License_Plate)
                custom_config = r'--oem 3 --psm 6'  # OCR Engine Mode 3 and Page Segmentation Mode 6 for a single block of text

                # Perform OCR on the cropped license plate image
                text = pytesseract.image_to_string(cropped_License_Plate, config=custom_config, lang='eng')

            if text != "":
                print("License plate is:", text)
                textDoc += text + "\n"

    # Show the current frame
    cv2.imshow("Current Frame", frame)

    # Save frames in a list
    framesList.append(frame)

    # Save transformed frames in an output video format
    outputVideo.write(frame)

# Write recognized license plates to a text file
with open('numberplates.txt', 'w') as f:
    f.write(textDoc)


### **Creating and Saving Video**

This Python code snippet performs several tasks related to video processing and text file manipulation:

1. **Releasing Video Output**: The code releases the video output using the `outputVideo.release()` function. This is typically done to free up system resources and ensure that the video is properly saved or closed.

2. **Fixing Video Output Codec**: It checks if a file specified by the `path` variable exists. If it does, the code removes the file using `os.remove(path)`. This step is likely intended to ensure that there are no conflicts or issues with the video output file in the specified path.

3. **Opening a Text File**: The code opens a text file named "numberplates.txt" in write mode using the `open()` function and assigns it to the `text_file` variable. This step prepares the file for writing text data.

4. **Writing Text to File**: It writes the contents of the `textDoc` variable (presumably a string containing text data) to the opened text file using the `text_file.write(textDoc)` method. This allows the text data to be saved to the "numberplates.txt" file.

5. **Closing the Text File**: Finally, the code closes the opened text file using `text_file.close()`. This step is important to ensure that any changes made to the file are saved and that system resources are released.

Overall, this code snippet is involved in video processing and text file handling, where it releases video output, manages video output file conflicts, and writes text data to a specified text file.


In [ ]:
#Releasing the video    
outputVideo.release()

# Fixing video output codec to run in the notebook\browser
if os.path.exists(path):
    os.remove(path)

#open text file
text_file = open("numberplates.txt", "w")
 
#write string to file
text_file.write(textDoc)
 
#close file
text_file.close()